In [ ]:
# ==========================================
# CELL 1: INSTALL & IMPORT
# ==========================================
# Kita install 'plotly' untuk grafik interaktif dan 'wordcloud'
!pip install pandas textblob plotly wordcloud matplotlib

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from datetime import datetime

# ==========================================
# CELL 2: LOAD DATA & PREPROCESSING
# ==========================================
def get_raw_url(github_url):
    if 'github.com' in github_url and '/blob/' in github_url:
        return github_url.replace('github.com', 'raw.githubusercontent.com').replace('/blob/', '/')
    return github_url

# === GANTI URL DI SINI ===
url_file_github = "https://github.com/rhnrafif/datamining_1/blob/main/data/yt_comment_ferry.json"

try:
    print("⏳ Sedang memuat data...")
    df = pd.read_json(get_raw_url(url_file_github))
    
    # 1. Parsing Tanggal (Penting untuk Time Series)
    df['published_at'] = pd.to_datetime(df['published_at'])
    df['date_only'] = df['published_at'].dt.date
    
    print(f"✅ Data Loaded: {len(df)} komentar.")

    # 2. Analisis Sentimen
    def get_score(text):
        try: return TextBlob(str(text)).sentiment.polarity
        except: return 0

    def get_label(score):
        if score > 0.1: return 'Positif'
        elif score < -0.1: return 'Negatif'
        else: return 'Netral'

    df['score'] = df['text'].apply(get_score)
    df['label'] = df['score'].apply(get_label)
    
    print("✅ Analisis Selesai.")

except Exception as e:
    print(f"❌ Error: {e}")

# ==========================================
# CELL 3: VISUALISASI 1 - PIE CHART & BAR CHART
# ==========================================
# Grafik Distribusi Total
if 'df' in locals():
    # 1. Pie Chart Interaktif
    fig_pie = px.pie(df, names='label', title='Proporsi Sentimen Komentar',
                     color='label', 
                     color_discrete_map={'Positif':'#00CC96', 'Negatif':'#EF553B', 'Netral':'#AB63FA'},
                     hole=0.4) # Donut chart style
    fig_pie.show()

    # 2. Bar Chart per Video (Top 5 Video Terkomentari)
    video_counts = df.groupby(['video_title', 'label']).size().reset_index(name='jumlah')
    # Filter hanya top 5 video biar grafik ga penuh
    top_videos = df['video_title'].value_counts().nlargest(5).index
    video_counts_filtered = video_counts[video_counts['video_title'].isin(top_videos)]

    fig_bar = px.bar(video_counts_filtered, x="jumlah", y="video_title", color="label",
                     title="Sentimen pada 5 Video Terpopuler", orientation='h',
                     color_discrete_map={'Positif':'#00CC96', 'Negatif':'#EF553B', 'Netral':'#AB63FA'})
    fig_bar.show()

# ==========================================
# CELL 4: VISUALISASI 2 - TIME SERIES (TREND)
# ==========================================
# Grafik Pergerakan Sentimen dari Waktu ke Waktu
if 'df' in locals():
    # Group by Date & Label
    timeline = df.groupby(['date_only', 'label']).size().reset_index(name='count')
    
    fig_line = px.line(timeline, x='date_only', y='count', color='label',
                       title='Tren Sentimen dari Waktu ke Waktu',
                       markers=True,
                       color_discrete_map={'Positif':'#00CC96', 'Negatif':'#EF553B', 'Netral':'#AB63FA'})
    
    # Update layout agar sumbu X enak dilihat
    fig_line.update_xaxes(title_text='Tanggal')
    fig_line.update_yaxes(title_text='Jumlah Komentar')
    fig_line.show()

# ==========================================
# CELL 5: VISUALISASI 3 - INTERACTIVE SCATTER PLOT
# ==========================================
# Hubungan antara Likes vs Sentimen (Apakah hate comment dapet banyak like?)
if 'df' in locals():
    fig_scat = px.scatter(df, x="score", y="likes", 
                          color="label", size="likes", 
                          hover_data=['text', 'author'], # Saat di-hover muncul isi komentarnya!
                          title="Hubungan Skor Sentimen vs Jumlah Likes",
                          color_discrete_map={'Positif':'#00CC96', 'Negatif':'#EF553B', 'Netral':'#AB63FA'})
    
    fig_scat.update_layout(xaxis_title="Skor Sentimen (-1 s.d 1)", yaxis_title="Jumlah Likes")
    fig_scat.show()

# ==========================================
# CELL 6: VISUALISASI 4 - WORD CLOUD
# ==========================================
# Melihat kata apa yang paling sering muncul
if 'df' in locals():
    # Gabungkan semua teks
    text_combined = " ".join(df['text'].astype(str).tolist()).lower()
    
    # Stopwords sederhana Indonesia (agar kata sambung tidak mendominasi)
    stopwords_indo = set([
        'dan', 'di', 'ke', 'dari', 'yang', 'ini', 'itu', 'aku', 'kamu', 'dia', 
        'pada', 'untuk', 'dengan', 'adalah', 'ya', 'bang', 'kak', 'nya', 'sih', 
        'dong', 'bisa', 'aja', 'saja', 'juga', 'sudah', 'tapi', 'kalau', 'kok'
    ])
    
    wordcloud = WordCloud(width=800, height=400, 
                          background_color='white', 
                          stopwords=stopwords_indo,
                          min_font_size=10).generate(text_combined)
    
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title("Word Cloud: Kata yang Sering Muncul")
    plt.show()

# ==========================================
# CELL 7: INSIGHT SUMMARY TABLE
# ==========================================
# Tabel Detail Komentar Ekstrem
if 'df' in locals():
    print("\n🔍 TOP 5 KOMENTAR PALING POSITIF:")
    # Sort by score tertinggi
    top_pos = df.sort_values(by='score', ascending=False).head(5)
    display(top_pos[['author', 'text', 'score', 'likes']])
    
    print("\n🔍 TOP 5 KOMENTAR PALING NEGATIF:")
    # Sort by score terendah
    top_neg = df.sort_values(by='score', ascending=True).head(5)
    display(top_neg[['author', 'text', 'score', 'likes']])

   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   ---------------------------------------- 624.3/624.3 kB 6.7 MB/s  0:00:00



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


⏳ Sedang memuat data...
❌ Error: HTTP Error 404: Not Found
